<a href="https://colab.research.google.com/github/idutsu/kirikuchikun-ai/blob/main/janome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install janome tqdm

In [ ]:
import os
import json
import glob
import html
import csv
from janome.tokenizer import Tokenizer
from tqdm import tqdm

In [ ]:
# Step 3: 形態素解析とパターン抽出の関数定義
def extract_noun_particle_verb(directory, output_csv, max_articles=None):
    """
    指定したディレクトリ内のすべてのJSONファイルから、
    名詞 + 助詞「を」 + 動詞のパターンにマッチする部分を抽出し、
    CSVファイルに書き出します。

    Parameters:
    - directory (str): JSONファイルが格納されているディレクトリのパス。
    - output_csv (str): 出力するCSVファイルのパス。
    - max_articles (int, optional): 処理する記事の最大数。Noneの場合は全ての記事を処理。
    """
    # Janomeのトークナイザーを初期化
    tokenizer = Tokenizer()

    # JSONファイルの検索
    json_files = glob.glob(os.path.join(directory, '**', '*.json'), recursive=True)

    if not json_files:
        print(f"指定されたディレクトリ内にJSONファイルが見つかりません: {directory}")
        return

    print(f"見つかったJSONファイルの数: {len(json_files)}")

    # CSVファイルの準備
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        # ヘッダーの書き込み
        csv_writer.writerow(['Article_ID', 'Title', 'Extracted_Phrase'])

        articles_processed = 0

        # 全JSONファイルをループ
        for json_file in tqdm(json_files, desc="JSONファイルの処理"):
            try:
                with open(json_file, 'r', encoding='utf-8') as f:
                    for line in f:
                        if max_articles is not None and articles_processed >= max_articles:
                            print("\n指定された記事数に達しました。処理を終了します。")
                            return

                        try:
                            article = json.loads(line)
                        except json.JSONDecodeError as e:
                            print(f"JSONの解析エラー: {e}")
                            continue

                        article_id = article.get('id', '')
                        title = article.get('title', '')
                        text = article.get('text', '')

                        # HTMLエンティティのデコード
                        decoded_text = html.unescape(text)

                        # 形態素解析
                        tokens = tokenizer.tokenize(decoded_text)

                        # パターンマッチングのためにトークンリストを作成
                        token_list = []
                        for token in tokens:
                            base_form = token.base_form
                            part_of_speech = token.part_of_speech.split(',')[0]
                            surface = token.surface
                            token_list.append((surface, part_of_speech, base_form))

                        # スライディングウィンドウでパターンを検索
                        for i in range(len(token_list) - 2):
                            first, second, third = token_list[i], token_list[i+1], token_list[i+2]

                            # パターン: 名詞 + 助詞「を」 + 動詞
                            if (first[1] == '名詞' and
                                second[1] == '助詞' and second[0] == 'を' and
                                third[1] == '動詞'):
                                # 抽出したフレーズを結合
                                phrase = first[0] + second[0] + third[0]
                                # CSVに書き出し
                                csv_writer.writerow([article_id, title, phrase])

                        articles_processed += 1

            except Exception as e:
                print(f"ファイル {json_file} の処理中にエラーが発生しました: {e}")
                continue

    print(f"\n処理が完了しました。抽出されたフレーズは {output_csv} に保存されています。")


In [ ]:

# 関数の実行
# JSONファイルが格納されているディレクトリのパスを指定
json_directory = "/content/drive/MyDrive/wikipedia_extracted_json"  # 実際のディレクトリパスに変更してください

# 出力するCSVファイルのパスを指定
output_csv_path = "/content/drive/MyDrive/extracted_phrases.csv"  # 実際の保存先パスに変更してください

# 処理する記事の最大数を指定（任意）
max_articles_to_process = None  # Noneの場合は全ての記事を処理します。例: 10000

# 関数を実行
extract_noun_particle_verb(json_directory, output_csv_path, max_articles_to_process)